# DATA CHALLENGE OWKIN
## Predicting lung cancer survival time
Using the following methods

### 1. DATA PRE-PROCESSING
1. Import the requried pacakges( _in our case "numpy","pandas","sklearn.modules","lifelines.modules" and "metrics_file"_ ) and the data.
2. Then clean the data and  gain insights.

In [1]:
import numpy as np 
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from metrics_t9gbvr2 import cindex

In [3]:
path='C:/Users/91970/Documents/MASTERS/YEAR 2/DATACHALLENGE_OWKIN/data/'
# Radiomic features(input_train)
radiomics_train = pd.read_csv(path+'radiomics_x_train.csv',header=[0,1],index_col=0,dtype="float64")
radiomics_train.columns=radiomics_train.columns.droplevel(0)
radiomics_train.describe()

,original_shape_Compactness1,original_shape_Compactness2,original_shape_Maximum3DDiameter,original_shape_SphericalDisproportion,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_Energy,original_firstorder_Entropy,...,original_glrlm_LongRunEmphasis,original_glrlm_GrayLevelNonUniformity,original_glrlm_RunLengthNonUniformity,original_glrlm_RunPercentage,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,3.000000e+02,300.000000,...,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.026078,0.255309,63.735090,1.666887,0.618682,10269.274116,0.332531,55941.000000,2.725407e+09,4232072,...,2.997984,4028.912972,21708.293730,0.770259,0.004949,1053.341229,0.004411,842.425719,0.009939,3956.614366
std,0.006213,0.114462,35.533323,0.321580,0.101082,11020.943961,0.165777,82370.656898,5.094067e+09,0.895564,...,2.277218,7354.203503,30741.811717,0.139863,0.008117,367.251140,0.006535,257.999681,0.025015,3721.271813
min,0.009288,0.030648,13.453624,1.154634,0.312946,409.699505,0.092706,640.000000,2.388461e+07,2.178197,...,1.129743,20.662759,503.361305,0.420690,0.000636,259.345076,0.000401,245.062735,0.001749,325.511805
25%,0.022028,0.172413,33.915016,1.443163,0.556575,2187.199167,0.200968,5586.750000,5.434917e+08,3.539630,...,1.380404,179.737494,3594.097996,0.653939,0.001541,790.854216,0.001333,657.397218,0.003539,1396.425566
50%,0.026071,0.241503,57.823002,1.605803,0.622741,5669.467145,0.296289,19523.000000,1.093244e+09,4.372496,...,2.183706,891.648859,10812.628268,0.793196,0.002709,1048.240239,0.002490,850.097187,0.004787,2566.288530
75%,0.030600,0.332701,82.933158,1.796703,0.692923,14084.942660,0.426285,68332.250000,2.994063e+09,5.017558,...,3.928845,4138.067422,28318.086695,0.903259,0.005065,1323.465451,0.004650,1017.454004,0.007573,5583.538887
max,0.042759,0.649631,216.716405,3.195442,0.866075,67799.783468,0.828710,502660.000000,5.101016e+10,5.585623,...,19.569205,61727.567864,267812.272378,0.960043,0.095065,1960.134253,0.069937,1621.293515,0.282310,25570.594397


In [4]:
# Clinical data(input_train)
clinical_data_train = pd.read_csv(path+'clinical_data_x_train.csv',index_col=0,usecols=['PatientID',
'Tstage', 'Mstage', 'Nstage', 'age'],dtype="float64")
clinical_data_train = clinical_data_train.fillna(clinical_data_train.mean()) #fill na values with mean
clinical_data_train.describe()

,Mstage,Nstage,Tstage,age
count,300.000000,300.000000,300.000000,300.000000
mean,0.040000,0.963333,2.103333,68.766856
std,0.314212,1.160378,1.072297,9.327554
min,0.000000,0.000000,1.000000,42.513300
25%,0.000000,0.000000,1.000000,63.117525
50%,0.000000,0.000000,2.000000,69.000000
75%,0.000000,2.000000,3.000000,75.942475
max,3.000000,3.000000,5.000000,87.126600


It can be observed that the count for age columns  is not the same as other columns which implies it has NA values. we can use mean,median or mode to replace those values.I have choosen mean to repalce the values.As any imputing performed on the training dataset will have to be performed on new data in the future when predictions are needed from the finalized model.I have made sure to do that in the further steps when required.

In [5]:
# Survival time(output_train)
y = pd.read_csv(path+'output_y_train.csv', index_col = 0,dtype="float64")
y.describe()

,SurvivalTime,Event
count,300.000000,300.00000
mean,867.056667,0.54000
std,723.538139,0.49923
min,14.000000,0.00000
25%,302.500000,0.00000
50%,644.000000,1.00000
75%,1298.500000,1.00000
max,3500.000000,1.00000


Concatnating the radiomics_train and clinical_data_train into one dataset named as x

In [6]:
x = pd.concat([radiomics_train, clinical_data_train], axis = 1)
x.describe()

,original_shape_Compactness1,original_shape_Compactness2,original_shape_Maximum3DDiameter,original_shape_SphericalDisproportion,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_Energy,original_firstorder_Entropy,...,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis,Mstage,Nstage,Tstage,age
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,3.000000e+02,300.000000,...,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.026078,0.255309,63.735090,1.666887,0.618682,10269.274116,0.332531,55941.000000,2.725407e+09,4.232072,...,0.004949,1053.341229,0.004411,842.425719,0.009939,3956.614366,0.040000,0.963333,2.103333,68.766856
std,0.006213,0.114462,35.533323,0.321580,0.101082,11020.943961,0.165777,82370.656898,5.094067e+09,0.895564,...,0.008117,367.251140,0.006535,257.999681,0.025015,3721.271813,0.314212,1.160378,1.072297,9.327554
min,0.009288,0.030648,13.453624,1.154634,0.312946,409.699505,0.092706,640.000000,2.388461e+07,2.178197,...,0.000636,259.345076,0.000401,245.062735,0.001749,325.511805,0.000000,0.000000,1.000000,42.513300
25%,0.022028,0.172413,33.915016,1.443163,0.556575,2187.199167,0.200968,5586.750000,5.434917e+08,3.539630,...,0.001541,790.854216,0.001333,657.397218,0.003539,1396.425566,0.000000,0.000000,1.000000,63.117525
50%,0.026071,0.241503,57.823002,1.605803,0.622741,5669.467145,0.296289,19523.000000,1.093244e+09,4.372496,...,0.002709,1048.240239,0.002490,850.097187,0.004787,2566.288530,0.000000,0.000000,2.000000,69.000000
75%,0.030600,0.332701,82.933158,1.796703,0.692923,14084.942660,0.426285,68332.250000,2.994063e+09,5.017558,...,0.005065,1323.465451,0.004650,1017.454004,0.007573,5583.538887,0.000000,2.000000,3.000000,75.942475
max,0.042759,0.649631,216.716405,3.195442,0.866075,67799.783468,0.828710,502660.000000,5.101016e+10,5.585623,...,0.095065,1960.134253,0.069937,1621.293515,0.282310,25570.594397,3.000000,3.000000,5.000000,87.126600


### 2.FEATURE SELECTION

Now that we have cleaned the data before building the model let's select the best features that are more relevent to the model form our training set x.


Let's write a function that can be used to split the data with based on specfic conditions other than the general conditions used in test_train_split and returns x_train,x_test,y_train,y_test

In [7]:
def get_x_y(x, y, train=0.75, selection=False, features=None) :
    """
    Parameters
    ----------
    x : pd.DataFrame
        DataFrame imported from the CSV for the radiomics
    y : pd.DataFrame
        DataFrame imported from the CSV for the survival times & events.
    train : 0<float<1
        Set the size of the training set,default is 0.75.
    selection : bool, optional
        In order to enable feature selection,default is False.
    features : pd.DataFreame.columns, optional
        The features to select. The default is None.
   

    Returns
    -------
    x_train, x_test, y_train, y_test : pd.DataFrame

    """
    x_train, x_test,y_train, y_test= train_test_split(x,y, train_size = train)
    if selection==True :
        x_train = x_train[features]
        x_test = x_test[features]
    else :
        x_train = x_train
        
    return(x_train, x_test, y_train, y_test)

#### FEATURE SELECTION 

I used  a two method step for feature selection:
1. Remove collinear features using pearson correlation
2. Remove features with low variances  so that the convergence does not  fail


In [8]:
def feature_selection(df1,df2,proba,thresh=0.0):
    """
    Parameters
    -----------
    df1 : pd.DataFrame
    df2 : pd.DataFrame
    proba : 0<float<1
           selection probaility for the pearson correlation
    thresh: 0<float<1 defualt is 0.0
          Select features with variances greater than thresh.
    
    
    Returns
    ------------
     selected_features: array of selected features
    """

    x_train, x_test, y_train, y_test=get_x_y(df1,df2)
    
    #Remove collinear features using pearson correlation
    corr=x_train.corr(method="pearson").abs()
    # selecting the uppertriangle in the correaltion matrix
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
    #dropping columns that have probability greater the given proba parameter
    drop = [column for column in upper.columns if any(upper[column] > proba)]
    x_train = x_train.drop(columns = drop)
    x_test = x_test.drop(columns = drop)
    #features after removing the correalted features
    features=x_train.columns     
    # iniating VarianceThreshold 
    vt = VarianceThreshold(threshold=thresh)
    # fit vt to x_trsin
    vt.fit(x_train)
    feature_indices = vt.get_support(indices=True)
    
    # remove low-variance columns from index
    feature_names = [features[idx]
                         for idx,_
                         in enumerate(features)
                         if idx
                         in feature_indices]

    # get the columns to be removed
    removed_features = np.setdiff1d(features,feature_names)
    print("Has {0} low-variance columns."
              .format(len(removed_features)),removed_features)
    #removing columns from the x_train
    x_train=x_train.drop(columns=removed_features)
    selcted_features=x_train.columns.values
    
    return selcted_features

### 3.PREDICTION

In [9]:
from sklearn.metrics import make_scorer
# to use the scoring functions in hyperparameter tuning and cross_validation
c_index = make_scorer(cindex,greater_is_better=True)

Buliding a model which is compatable for scikit learn which can be further used for crossvalidation and  hyperparametric tuning 

In [10]:
from sklearn.base import BaseEstimator
class Coxph(BaseEstimator):
    """
    Parameters
    ------------------
    
    penalizer:float, optional (default=0.0)
        Attachs a L2 penalizer to the size of the coefficients during regression and also improves
        stability of the estimates and controls for high correlation between covariates.
    
    alpha:float,(default=0.05)
        the level in the confidence intervals
    
    output_function: types of estimator,(default=cox)
    
    Returns:
    ---------
    Coxph fitter with penalizer and alpha as parameters which is scikit compatabile 
    
    """

    def __init__(self, penalizer=0.0,alpha=0.05,output_function="cox"):
        self.penalizer = penalizer
        self.alpha = alpha

        if output_function == 'cox':
            self.output_function = CoxPHFitter(penalizer=self.penalizer,alpha=self.alpha)
        else:
            self.output_function = output_function

        return 


    def fit(self, x, y):
        """
        parameters:
        ------------------
        x : array-like of shape (n_samples, n_features)
            Training vector, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like of shape (n_samples, n_output) or (n_samples,)
    
        Returns:
        fit the estimator with the sets of parameters
        ------------
        """
        return self.output_function.fit(pd.concat([x, y], axis = 1),duration_col = 'SurvivalTime', event_col='Event')
    
    def predict(self, x):
        """
       Parameters
       ----------
       x : pd.DataFrame
           Dataframe to which output is predicted
       Returns
       ----------
       y_pred : pd.DataFrame
                DataFrame with the predicted output values
                which  is one f the input to the score function
       """
        
        y_pred=self.output_function.predict_expectation(x)
        y_pred.columns = ['SurvivalTime']
        y_pred.index.names = ['PatientID']
        y_pred.index = x.index
        y_pred['Event'] = None
        return y_pred
    
    def score(self,a,b):
        """
        Parameters:
        -------------
        a:pd.DataFrame
        
        b:pd.DataFrame
        
        Returns:
        -------------
        score: float
               using the metric cindex
        """
        score= cindex(a,b)
        return score 
    
    def summary(self,x,y,n):
        """
        Parameters:
        -----------
        x:pd.dataframe
        y:pd.dataframe
        n: int
           specify the number of decimal places to show

        Returns:
        ----------
        summary: Print summary statistics describing the fit, the coefficients, and the error bounds.
        """
        summary= self.fit(x,y).print_summary(n)
        return summary
    
    def coeff(self,x,y):
        """
        Parameters:
        ------------
        x:pd.dataframe
        
        y:pd.dataframe
        
        Returns:
        -----------
        coeff : Series
        The estimated coefficients.`
        """
        coeff =self.fit(x,y).hazards_
        return coeff
    

    def get_params(self, deep=True):
        return {"penalizer": self.penalizer,
                "output_function":self.output_function}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

#### 3.1 CROSS VALIDATION

let's cross_validate the model and get the best proba for the model

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
proba=[0.8,0.85,0.9]  
p=np.logspace(0,3,10)
scores=[]
cv= KFold(n_splits=5)
for j in range(len(proba)):
    f=feature_selection(x,y,proba=proba[j],thresh=0.1)
    x_train,x_test,y_train,y_test=get_x_y(x,y,selection=True,features=f)
    for i in range(len(p)):
        model=Coxph(output_function="cox",penalizer=p[i])
        s=np.mean(cross_val_score(model,x_train,y_train, scoring = c_index,cv=cv))
        scores.append(s)

Has 7 low-variance columns. ['Mstage' 'original_glcm_Correlation' 'original_glcm_Imc1'
 'original_glcm_Imc2' 'original_glrlm_LowGrayLevelRunEmphasis'
 'original_shape_Compactness1' 'original_shape_SurfaceVolumeRatio']
Has 7 low-variance columns. ['original_glcm_Correlation' 'original_glcm_Imc1' 'original_glcm_Imc2'
 'original_glrlm_LongRunLowGrayLevelEmphasis'
 'original_glrlm_LowGrayLevelRunEmphasis' 'original_shape_Compactness1'
 'original_shape_SurfaceVolumeRatio']
Has 9 low-variance columns. ['Mstage' 'original_glcm_Correlation' 'original_glcm_Idmn'
 'original_glcm_Imc1' 'original_glcm_Imc2'
 'original_glrlm_LongRunLowGrayLevelEmphasis'
 'original_glrlm_LowGrayLevelRunEmphasis' 'original_shape_Compactness1'
 'original_shape_SurfaceVolumeRatio']


In [12]:
#creating a data frame with the scores
header=pd.MultiIndex.from_product([proba,p])
score=pd.DataFrame(scores,index=header).T
print(score[0.8][p].mean(axis=1),score[0.85][p].mean(axis=1),score[0.9][p].mean(axis=1))
score

0    0.683998
dtype: float64 0    0.664302
dtype: float64 0    0.667675
dtype: float64


0.8                                                              \
  1.000000    2.154435    4.641589    10.000000   21.544347   46.415888     
0    0.684284    0.685439    0.683563    0.684043    0.683679    0.682918   

                                                   ...         0.9  \
  100.000000  215.443469  464.158883  1000.000000  ... 1.000000      
0    0.682921     0.68652    0.683315    0.683295  ...    0.676871   

                                                                           \
  2.154435    4.641589    10.000000   21.544347   46.415888   100.000000    
0    0.675918    0.679299    0.677628    0.675369    0.671076    0.661843   

                                       
  215.443469  464.158883  1000.000000  
0    0.657487    0.650314    0.650944  

[1 rows x 30 columns]

After the cross validation we can see that proba with 0.85 has the highest score so we chose proba =0.85 as a standard for our model

In [13]:
#Spliting the data into train test after the feature selection
f=feature_selection(x,y,proba=0.85,thresh=0.1)
x_train,x_test,y_train,y_test=get_x_y(x,y,selection=True,features=f)

Has 8 low-variance columns. ['Mstage' 'original_glcm_Correlation' 'original_glcm_Idmn'
 'original_glcm_Imc1' 'original_glcm_Imc2'
 'original_glrlm_LowGrayLevelRunEmphasis' 'original_shape_Compactness1'
 'original_shape_SurfaceVolumeRatio']


#### 3.2 HYPERPARAMETER TUNING

In order to tune the parameters with and get the best estimator i have used gridsearchCV and Kfold cross validation

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid={ 'penalizer': np.logspace(0,3,10)}
cv= KFold(n_splits=5)
clf=GridSearchCV(estimator=Coxph(output_function="cox"),param_grid=param_grid,cv=cv,scoring=c_index)
#fit the data 
clf.fit(x_train,y_train)
# best _estimator
clf.best_estimator_

Coxph(output_function=<lifelines.CoxPHFitter: fitted with 225 observations, 103 censored>,
      penalizer=1.0)

Fitting the model with the best parameters which is obtained after the grid search i.e penalizer=1.0

In [14]:
model=Coxph(output_function="cox",penalizer=1.0)
model.fit(x_train,y_train)
#summary
model.summary(x_train,y_train,3)

<lifelines.CoxPHFitter: fitted with 225 observations, 103 censored>
      duration col = 'SurvivalTime'
         event col = 'Event'
number of subjects = 225
  number of events = 122
    log-likelihood = -562.525
  time fit was run = 2020-02-02 16:50:18 UTC

---
                                        coef  exp(coef)  se(coef)      z       p  log(p)  lower 0.05  upper 0.05    
original_shape_Maximum3DDiameter      -0.002      0.998     0.007 -0.274   0.784  -0.243      -0.002      -0.001    
original_shape_SurfaceArea            -0.000      1.000     0.000 -0.172   0.863  -0.147      -0.000      -0.000    
original_firstorder_Energy            -0.000      1.000     0.000 -1.460   0.144  -1.937      -0.000      -0.000    
original_firstorder_Entropy           -0.560      0.571     0.319 -1.758   0.079  -2.541      -0.580      -0.540    
original_firstorder_Kurtosis           0.005      1.005     0.010  0.483   0.629  -0.463       0.004       0.005    
original_firstorder_Maximum        

We can see a lot of coef values are zero as  they contribute nothig we will reomve them

In [15]:
#getting the coef from the model
coef=model.coeff(x_train,y_train).round(decimals=3)
mask = coef.eq(0.0)
#removing the particular columns with zero values from the data
best_features=coef.loc[~mask.all(1), ~mask.all()].columns


fiting the model with features selected after removong the columns with coef values =0.0

In [16]:
model=Coxph(output_function="cox",penalizer=1)
model.fit(x_train[best_features],y_train)
#perdiction using best features
y_predict=model.predict(x_test[best_features])

### 4.SUBMISSION

In [20]:
radiomics_test = pd.read_csv(path+'radiomics_x_test.csv',header=[0,1],index_col=0)
radiomics_test.columns=radiomics_test.columns.droplevel(0)
# Clinical data
clinical_data_test = pd.read_csv(path+'clinical_data_x_test.csv',index_col=0,usecols=['PatientID','Tstage', 'Mstage', 'Nstage', 'age'])
clinical_data_test = clinical_data_test.fillna(clinical_data_train.mean())

# predicyions using best features
test = pd.concat([radiomics_test, clinical_data_test], axis = 1)
test = test.fillna(x_test.mean()) # Fill Na values (age)
test=test[best_features]

# Testing the model
lifetime_pred = model.predict(test)

# Prediction
lifetime_pred.to_csv(path+'y_final.csv')


**The final score after submission is 0.7248**